# --- Day 19: Go With The Flow ---

With the Elves well on their way constructing the North Pole base, you turn your attention back to understanding the inner workings of programming the device.

You can't help but notice that the device's opcodes don't contain any flow control like jump instructions. The device's manual goes on to explain:

"In programs where flow control is required, the instruction pointer can be bound to a register so that it can be manipulated directly. This way, `setr/seti` can function as absolute jumps, `addr/addi` can function as relative jumps, and other opcodes can cause truly fascinating effects."

This mechanism is achieved through a declaration like `#ip 1`, which would modify register 1 so that accesses to it let the program indirectly access the instruction pointer itself. To compensate for this kind of binding, there are now six registers (numbered 0 through 5); the five not bound to the instruction pointer behave as normal. Otherwise, the same rules apply as the last time you worked with this device.

When the instruction pointer is bound to a register, its value is written to that register just before each instruction is executed, and the value of that register is written back to the instruction pointer immediately after each instruction finishes execution. Afterward, move to the next instruction by adding one to the instruction pointer, even if the value in the instruction pointer was just updated by an instruction. (Because of this, instructions must effectively set the instruction pointer to the instruction before the one they want executed next.)

The instruction pointer is 0 during the first instruction, 1 during the second, and so on. If the instruction pointer ever causes the device to attempt to load an instruction outside the instructions defined in the program, the program instead immediately halts. The instruction pointer starts at 0.

It turns out that this new information is already proving useful: the CPU in the device is not very powerful, and a background process is occupying most of its time. You dump the background process' declarations and instructions to a file (your puzzle input), making sure to use the names of the opcodes rather than the numbers.

For example, suppose you have the following program:
```
#ip 0
seti 5 0 1
seti 6 0 2
addi 0 1 0
addr 1 2 3
setr 1 0 0
seti 8 0 4
seti 9 0 5
```
When executed, the following instructions are executed. Each line contains the value of the instruction pointer at the time the instruction started, the values of the six registers before executing the instructions (in square brackets), the instruction itself, and the values of the six registers after executing the instruction (also in square brackets).
```
ip=0 [0, 0, 0, 0, 0, 0] seti 5 0 1 [0, 5, 0, 0, 0, 0]
ip=1 [1, 5, 0, 0, 0, 0] seti 6 0 2 [1, 5, 6, 0, 0, 0]
ip=2 [2, 5, 6, 0, 0, 0] addi 0 1 0 [3, 5, 6, 0, 0, 0]
ip=4 [4, 5, 6, 0, 0, 0] setr 1 0 0 [5, 5, 6, 0, 0, 0]
ip=6 [6, 5, 6, 0, 0, 0] seti 9 0 5 [6, 5, 6, 0, 0, 9]
```
In detail, when running this program, the following events occur:

* The first line (`#ip 0`) indicates that the instruction pointer should be bound to register 0 in this program. This is not an instruction, and so the value of the instruction pointer does not change during the processing of this line.
*    The instruction pointer contains 0, and so the first instruction is executed (`seti 5 0 1`). It updates register 0 to the current instruction pointer value (0), sets register 1 to 5, sets the instruction pointer to the value of register 0 (which has no effect, as the instruction did not modify register 0), and then adds one to the instruction pointer.
*    The instruction pointer contains 1, and so the second instruction, `seti 6 0 2`, is executed. This is very similar to the instruction before it: 6 is stored in register 2, and the instruction pointer is left with the value 2.
*    The instruction pointer is 2, which points at the instruction `addi 0 1 0`. This is like a relative jump: the value of the instruction pointer, 2, is loaded into register 0. Then, addi finds the result of adding the value in register 0 and the value 1, storing the result, 3, back in register 0. Register 0 is then copied back to the instruction pointer, which will cause it to end up 1 larger than it would have otherwise and skip the next instruction (`addr 1 2 3`) entirely. Finally, 1 is added to the instruction pointer.
*    The instruction pointer is 4, so the instruction `setr 1 0 0` is run. This is like an absolute jump: it copies the value contained in register 1, 5, into register 0, which causes it to end up in the instruction pointer. The instruction pointer is then incremented, leaving it at 6.
*    The instruction pointer is 6, so the instruction `seti 9 0 5` stores 9 into register 5. The instruction pointer is incremented, causing it to point outside the program, and so the program ends.

**What value is left in register 0 when the background process halts?**


In [1]:
import re

INSTRS = ['addr', 'addi', 'mulr', 'muli', 'banr', 'bani', 'borr', 'bori', 'setr', 'seti', 'gtir', 'gtri', 'gtrr', 'eqir', 'eqri', 'eqrr']

instRE = re.compile(r'([a-z]{4}) (\d+) (\d+) (\d+)')
directiveRE = re.compile(r'#ip (\d+)')

class VMExecutionStop(Exception):
    def __init__(self, *args, **kwargs):
        Exception.__init__(self,*args,**kwargs)

class VM(object):
    
    def __init__(self, prog, nregisters=6, debug=False):
        self.prog = prog[:] # list of strings
        self.ip = 0 # instruction pointer
        self.nregisters = nregisters
        self.registers = [0] * nregisters
        self.debug = debug
        self.ip_register = None
        self.steps = 0
        self._run_preprocessor()
        
    # == CORE ROUTINES =============================================================================
    
    def parse(self, line):
        '''Splits program line into instruction name and arguments, converting the args to int()'''
        match = re.search(instRE, line)
        if match:
            instr, *args = match.groups()
            args = list(map(int, args))
        else:
            raise ValueError(f'Malformed program line: \'{line}\'')
        return instr, args
    
    def dispatch(self, instr, A, B, C):
        '''Runs the appropriate method function with the arguments A, B, C'''
        if instr not in INSTRS:
            raise RuntimeError(f'No such instruction {instr}')
        VM.__dict__[instr](self, A, B, C)
        
    def step(self):
        '''Executes the instruction at the instruction pointer, ip, maintain the instruction pointer and any defined #ip register'''
        # test for program completion
        if self.ip < 0 or self.ip >= len(self.prog):
            raise VMExecutionStop(f'Finished execution of program at address {self.ip} after {self.steps} steps. Registers are {self.registers}.')
            
        # When the instruction pointer is bound to a register, its value is written to that register just before each instruction is executed
        if self.ip_register is not None:
            self.registers[self.ip_register] = self.ip
        
        # Parse and dispatch
        line = self.prog[self.ip]
        instr, (A, B, C) = self.parse(line)
        self.dispatch(instr, A, B, C)
        
        # the value of that register is written back to the instruction pointer immediately after each instruction finishes execution
        if self.ip_register is not None:
            self.ip = self.registers[self.ip_register]
        # 
        self.steps += 1
        self.ip += 1
        
        
    def run(self):
        '''Runs the program until completion'''
        while True:
            try:
                self.step()
            except VMExecutionStop as e:
                print(e)
                return self.registers[0]
        
    # == UTILITIES =================================================================================
    
    def _run_preprocessor(self):
        processed = []
        for line in self.prog:
            directive_match = re.search(directiveRE, line)
            if directive_match:
                self.ip_register = int(directive_match.groups()[0])
            else:
                processed.append(line)
        self.prog = processed
    
    def _set_registers(self, l):
        if len(l) != self.nregisters:
            raise ValueError(f'Wrong number of register values provided - got {len(l)}, should be {self.nregisters}')
        self.registers = l[:]
    
    # == INTROSPECTION =============================================================================
    
    def __str__(self):
        out = f'VM: IP={self.ip}'
        if self.ip < 0 or self.ip >= len(self.prog):
            out += ' HALTED'
        else:
            out += f' Current Instr: {self.prog[self.ip]}'
        out += f'\nIP Register: {repr(self.ip_register)}'
        out += f'\nRegister values: {self.registers}'
        return out
        
    __repr__ = __str__
    
    def dump(self):
        print(str(self))
    
    # == INSTRUCTIONS ==============================================================================
    
    # Addition:
   
    def addr(self, A, B, C):
        # addr (add register) stores into register C the result of adding register A and register B.
        self.registers[C] = self.registers[A] + self.registers[B]
          
    def addi(self, A, B, C):
        # addi (add immediate) stores into register C the result of adding register A and value B.
        self.registers[C] = self.registers[A] + B

    # Multiplication:
        
    def mulr(self, A, B, C):
        # mulr (multiply register) stores into register C the result of multiplying register A and register B.
        self.registers[C] = self.registers[A] * self.registers[B]
        
    def muli(self, A, B, C):
        # muli (multiply immediate) stores into register C the result of multiplying register A and value B.
        self.registers[C] = self.registers[A] * B

    # Bitwise AND:
        
    def banr(self, A, B, C):
        # banr (bitwise AND register) stores into register C the result of the bitwise AND of register A and register B.
        self.registers[C] = self.registers[A] & self.registers[B]
        
    def bani(self, A, B, C):
        # bani (bitwise AND immediate) stores into register C the result of the bitwise AND of register A and value B.
        self.registers[C] = self.registers[A] & B
        
    # Bitwise OR:
    
    def borr(self, A, B, C):
        # borr (bitwise OR register) stores into register C the result of the bitwise OR of register A and register B.
        self.registers[C] = self.registers[A] | self.registers[B]
        
    def bori(self, A, B, C):
        # bori (bitwise OR immediate) stores into register C the result of the bitwise OR of register A and value B.
        self.registers[C] = self.registers[A] | B

    # Assignment:

    def setr(self, A, B, C):
        # setr (set register) copies the contents of register A into register C. (Input B is ignored.)
        self.registers[C] = self.registers[A]
    
    def seti(self, A, B, C):
        # seti (set immediate) stores value A into register C. (Input B is ignored.)
        self.registers[C] = A
        
    # Greater-than testing:

    def gtir(self, A, B, C):
        # gtir (greater-than immediate/register) sets register C to 1 if value A is greater than register B. Otherwise, register C is set to 0.
        self.registers[C] = 1 if A > self.registers[B] else 0
        
    def gtri(self, A, B, C):
        # gtri (greater-than register/immediate) sets register C to 1 if register A is greater than value B. Otherwise, register C is set to 0.
        self.registers[C] = 1 if self.registers[A] > B else 0
        
    def gtrr(self, A, B, C):
        # gtrr (greater-than register/register) sets register C to 1 if register A is greater than register B. Otherwise, register C is set to 0.
        self.registers[C] = 1 if self.registers[A] > self.registers[B] else 0
        
    # Equality testing:

    def eqir(self, A, B, C):
        # eqir (equal immediate/register) sets register C to 1 if value A is equal to register B. Otherwise, register C is set to 0.
        self.registers[C] = 1 if A == self.registers[B] else 0
        
    def eqri(self, A, B, C):
        # eqri (equal register/immediate) sets register C to 1 if register A is equal to value B. Otherwise, register C is set to 0.
        self.registers[C] = 1 if self.registers[A] == B else 0
        
    def eqrr(self, A, B, C):
        # eqrr (equal register/register) sets register C to 1 if register A is equal to register B. Otherwise, register C is set to 0.
        self.registers[C] = 1 if self.registers[A] == self.registers[B] else 0


In [2]:
# wrapping blocks in __name__ == '__main__' so that I can export to .py and use the VM as an import in scripts if necessary
if __name__ == '__main__':
    
    real_prog = open('input.txt').read().splitlines()

    test_prog = '''#ip 0
    seti 5 0 1
    seti 6 0 2
    addi 0 1 0
    addr 1 2 3
    setr 1 0 0
    seti 8 0 4
    seti 9 0 5'''.splitlines()
    
    print('Running test program ...')
    assert(VM(test_prog).run() == 6)
    print('... done')

    p1vm = VM(real_prog)
    part1 = p1vm.run()
  
    print(f'Part 1 answer: {part1}')


Running test program ...
Finished execution of program at address 7 after 5 steps. Registers are [6, 5, 6, 0, 0, 9].
... done
Finished execution of program at address 257 after 6643024 steps. Registers are [912, 911, 912, 1, 256, 912].
Part 1 answer: 912


# Part 2

In [3]:
if __name__ == '__main__':
    if False:
        # wishful thinking...
        p2vm = VM(real_prog)
        p2vm._set_registers([1, 0, 0, 0, 0, 0])

        p2vm.run()


## Annotating the asm code a bit


### start by jumping to init()
```
 0: addi 4 16 4 # jump rel +17 => 17:
``` 
### main() code:
```
 1: seti 1  7 2 # r2 = 1
 2: seti 1  1 5 # r5 = 1
 3: mulr 2  5 3 # r3 = r2 * r5
 4: eqrr 3  1 3 # if r1 == r3: r3 = 1 else r3 = 0
 5: addr 3  4 4 # jump rel +(r3 + 1) 
 6: addi 4  1 4 # jump rel +2 => 8: 
 7: addr 2  0 0 # r0 = r0 + r2
 8: addi 5  1 5 # r5 = r5 + 1
 9: gtrr 5  1 3 # if r5 > r1: r3 = 1 else r3 = 0
10: addr 4  3 4 # jump rel +(r3 + 1)
11: seti 2  7 4 # jump to 3:
12: addi 2  1 2 # r2 = r2 + 1
13: gtrr 2  1 3 # if r2 > r1: r3 = 1 else r3 = 0
14: addr 3  4 4 # jump rel +(r3 + 1)
15: seti 1  3 4 # jump to 2
16: mulr 4  4 4 # jump to 256 => HALT
```
### init() code: 
#### (I'm not annotating this since empirically, I can see what the result is - for part 1 r1 gets set to 911 and for part 2 r1 gets set to 10551311)
```
17: addi 1  2 1
18: mulr 1  1 1
19: mulr 4  1 1
20: muli 1 11 1
21: addi 3  3 3
22: mulr 3  4 3
23: addi 3  9 3
24: addr 1  3 1
25: addr 4  0 4
26: seti 0  1 4
27: setr 4  9 3
28: mulr 3  4 3
29: addr 4  3 3
30: mulr 4  3 3
31: muli 3 14 3
32: mulr 3  4 3
33: addr 1  3 1
34: seti 0  6 0 # r0 = 0
35: seti 0  7 4 # jump to r0 + 1 (=> 1: - main())
```

## rewriting the main() code

```
 1: r2 = 1
 2: r5 = 1
 3: r3 = r2 * r5
 4: if r1 == r3: 
        r0 = r0 + r2
 8: r5 = r5 + 1
 9: if r5 > r1:
        r2 = r2 + 1
11: jump to 3:
13: if r2 > r1: 
        HALT
    else:
        jump to 2
```        
        
## Rewriting as Python gives:      

```Python        
r0 = 0
r1 = 10551311
for r5 in range(1, r1 + 1):
    for r2 in range(1, r1 + 1):
        if r2 * r5 == r1:
            r0 += r2
```

i.e. we are calculating the sum of the factors of the number that r1 gets initialised with

testing this with 911 (the initialisation value for part 1) gives:

In [4]:
if __name__ == '__main__':
    r0 = 0
    r1 = 911
    for r5 in range(1, r1 + 1):
        for r2 in range(1, r1 + 1):
            if r2 * r5 == r1:
                r0 += r2
    print(f'Testing reverse-engineered code with part 1 values (value should be 912): {r0}')

Testing reverse-engineered code with part 1 values (value should be 912): 912


In [5]:
if __name__ == '__main__':
    from math import sqrt

    def divisors(N):
        d = [1]
        for i in range(2, int(sqrt(N))):
            if N % i == 0:
                d.append(i)
                d.append(N // i)
        d.append(N)
        return sorted(d)

    print(f' Part 2 answer: {sum(divisors(10551311))}')

 Part 2 answer: 10576224
